In [ ]:
import pandas as pd
import re

In [ ]:
f = open('WhatsApp Chat with Subject.txt', 'r', encoding='utf-8')

In [ ]:
data = f.read()

In [ ]:
# print(data)

pattern = r'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s'

In [ ]:
messages = re.split(pattern, data)[1:]
# messages
len(messages)

In [ ]:
dates = re.findall(pattern, data)
# dates
len(dates)

In [ ]:
df = pd.DataFrame({'user_message': messages, 'message_date': dates})

# convert messsage_date type
df['message_date'] = pd.to_datetime(df['message_date'], format='%d/%m/%Y, %H:%M - ')

df.rename(columns = {'message_date': 'date'}, inplace = True)
df.head()

In [ ]:
df.shape

In [ ]:
# separate users and messages
users = []
messages = []
for message in df['user_message']:
    entry = re.split(r'([\w\W]+?):\s', message)
    # print(entry)
    if entry[1:]: # username
        users.append(entry[1])
        messages.append(entry[2])
    else:
        users.append('whatsapp_notification')
        messages.append(entry[0])

# print (len(messages))
df['user'] = users
df['message'] = messages
df.drop(columns=['user_message'], inplace=True)

df.head()

In [ ]:
df['year'] = df['date'].dt.year

In [ ]:
df['month'] = df['date'].dt.month_name()

In [ ]:
df['day'] = df['date'].dt.day

In [ ]:
df['hour'] = df['date'].dt.hour

In [ ]:
df['minute'] = df['date'].dt.minute

In [ ]:
df.head()

In [ ]:
df[df['user'] == 'Ayushi'].shape

In [ ]:
words = []
for message in df['message']:
    words.extend(message.split())

In [ ]:
len(words)